# Visualizing Satellite Data Over Time

Use branca for linear colormap

In [ ]:
import geopandas as gpd
import folium
import branca
from folium.plugins import TimestampedGeoJson

In [ ]:
directory = '../DOAE spatial data/ดาวเทียม/'
rice_file = directory + '20200229_16_Rice/rice_20200229.shp'

In [ ]:
rice = gpd.read_file(rice_file, encoding='utf-8')

In [ ]:
rice.columns

In [ ]:
rice.iloc[0].geometry

In [ ]:
rice.iloc[0].geometry.__geo_interface__

In [ ]:
rice.iloc[0]['harv_date']

In [ ]:
rice.iloc[1].geometry

In [ ]:
len(rice.iloc[1].geometry.__geo_interface__['coordinates'])

In [ ]:
def get_polygon(row):
    return {
        'type': 'Feature',
        'geometry': row.geometry.__geo_interface__,
        'properties': {
            'style': {
                'color': 'blue',
            },
            'times': [row['harv_date']]*len(row.geometry.__geo_interface__['coordinates'])
        }
    }

In [ ]:
MAX_RECORDS = 1000

In [ ]:
rice[:MAX_RECORDS]['harv_date'].unique()

In [ ]:
polygons = []

for i, row in rice[:MAX_RECORDS].iterrows():
    polygons.append(get_polygon(row))

In [ ]:
m = folium.Map(location=[14.069446, 100.603271], zoom_start=6)

TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': polygons},
    period='P1M',
    duration='P16D',
    auto_play=False,
    loop=False,
    loop_button=True,
    date_options='YYYY/MM/DD',
).add_to(m)
    
m

## Add Color based on yields

In [ ]:
colormap = branca.colormap.LinearColormap(vmin=rice[:MAX_RECORDS]['yield'].min(), 
                                          vmax=rice[:MAX_RECORDS]['yield'].max(), 
                                          colors=['yellow','red'],
                                          caption="Rice Yield")

In [ ]:
def get_polygon_color(row):
    return {
        'type': 'Feature',
        'geometry': row.geometry.__geo_interface__,
        'properties': {
            'style': {
                'color': colormap(row['yield']),
            },
            'times': [row['harv_date']]*len(row.geometry.__geo_interface__['coordinates'])
        }
    }

In [ ]:
color_polygons = []

for i, row in rice[:MAX_RECORDS].iterrows():
    color_polygons.append(get_polygon_color(row))

In [ ]:
m = folium.Map(location=[14.069446, 100.603271], zoom_start=6)

TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': color_polygons},
    period='P1M',
    duration='P16D',
    auto_play=False,
    loop=False,
    loop_button=True,
    date_options='YYYY/MM/DD',
).add_to(m)
m.add_child(colormap)
m

In [ ]:
m.save('output/sat_viz_time.html')